In [ ]:
library(class)

In [12]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

In [ ]:
f <- read.csv("../datasets/wbc_csv.csv")
head(f)

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave.points_mean,⋯,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave.points_worst,symmetry_worst,fractal_dimension_worst
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,⋯,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
2,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,⋯,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
3,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,⋯,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
4,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,⋯,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
5,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,⋯,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678
6,843786,M,12.45,15.70,82.57,477.1,0.12780,0.17000,0.1578,0.08089,⋯,15.47,23.75,103.40,741.6,0.1791,0.5249,0.5355,0.1741,0.3985,0.12440


In [3]:
df=pd.read_csv("../datasets/wbc_csv.csv")
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [ ]:
f$diagnosis <- as.factor(f$diagnosis)

In [4]:
# Assuming f is a pandas DataFrame
df['diagnosis'] = df['diagnosis'].astype('category')

In [ ]:
# Shuffle Data
set.seed(123)
f <- f[sample(nrow(f)), ]

In [5]:
# Shuffle DataFrame `f` with a fixed random seed
df = df.sample(frac=1, random_state=123).reset_index(drop=True)

In [ ]:
n <- function(b)
    {
        (b - min(b)) / (max(b) - min(b))
    }

In [6]:
def n(b):
    return (b - np.min(b)) / (np.max(b) - np.min(b))

In [ ]:
# apply func on 3 to 32 columns
f1 <- f[, 3 : 32]
fnor <- as.data.frame(lapply(f1, n))

In [7]:
# Select columns 3 to 32 (Python uses 0-based indexing)
df1 = df.iloc[:, 2:32]  # 2:32 selects columns 3 to 32

# Apply normalization function to each column
dfnor = df1.apply(n)  # Assuming n() is already defined

In [ ]:
head(fnor)

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave.points_mean,symmetry_mean,fractal_dimension_mean,⋯,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave.points_worst,symmetry_worst,fractal_dimension_worst
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.38567845,0.6797430,0.3656969,0.24432662,0.27597725,0.0818048,0.109793814,0.136133201,0.4000000,0.06276327,⋯,0.3319104,0.6633795,0.29727576,0.18339560,0.28811992,0.069243531,0.123562300,0.22594502,0.3287995,0.04335563
2,0.35112878,0.5843761,0.3348766,0.21319194,0.15636003,0.1007607,0.081443299,0.086332008,0.3267677,0.09203875,⋯,0.2657417,0.5319829,0.24896658,0.13502753,0.20161131,0.115173036,0.117571885,0.19116838,0.1537552,0.06296734
3,0.28534242,0.4230639,0.2641144,0.16241782,0.08919383,0.0000000,0.003737113,0.009204771,0.1691919,0.05012637,⋯,0.2159374,0.4530917,0.18810698,0.10410932,0.06656541,0.006820541,0.006371406,0.03181787,0.1438991,0.02223534
4,0.07525202,0.1146432,0.0742174,0.03300106,0.46014264,0.1746519,0.060098407,0.075049702,0.3121212,0.44860994,⋯,0.0548915,0.1713753,0.06419642,0.02221785,0.61368289,0.190363924,0.140095847,0.29250859,0.2795190,0.32703660
5,0.37289981,0.4565438,0.3911962,0.22392365,0.46736481,0.5478805,0.397610122,0.441252485,0.3419192,0.35299073,⋯,0.2899324,0.4200426,0.33960855,0.14734074,0.39906227,0.424474391,0.366533546,0.59347079,0.2828701,0.20969435
6,0.37337309,0.2353737,0.3790339,0.22863203,0.57389185,0.4463530,0.395970009,0.443489066,0.5540404,0.37320977,⋯,0.3870509,0.4091151,0.38194133,0.22360401,0.77019085,0.408174947,0.375718850,0.69656357,0.3981865,0.33753116


In [9]:
dfnor.head(6)

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,0.202045,0.171458,0.190657,0.104560,0.274713,0.077296,0.002281,0.014617,0.360101,0.228517,...,0.171825,0.267591,0.157727,0.075575,0.300007,0.068545,0.004407,0.057285,0.246403,0.125541
1,0.130673,0.201556,0.122383,0.062100,0.340706,0.084381,0.025843,0.069930,0.515657,0.277169,...,0.100320,0.236141,0.088052,0.041781,0.498778,0.064431,0.032292,0.177285,0.251528,0.175193
2,0.499740,0.324992,0.492779,0.342778,0.334477,0.308018,0.242737,0.372167,0.225253,0.104254,...,0.444326,0.368337,0.443697,0.258995,0.441986,0.305333,0.280192,0.666323,0.268677,0.154991
3,0.285342,0.423064,0.264114,0.162418,0.089194,0.000000,0.003737,0.009205,0.169192,0.050126,...,0.215937,0.453092,0.188107,0.104109,0.066565,0.006821,0.006371,0.031818,0.143899,0.022235
4,0.543282,0.297937,0.534241,0.395122,0.416268,0.263542,0.312793,0.437127,0.541414,0.216091,...,0.533618,0.417377,0.506948,0.348457,0.453213,0.176199,0.252157,0.564261,0.419870,0.201692
5,0.381419,0.237741,0.379656,0.231559,0.417080,0.358935,0.180904,0.305268,0.307071,0.394482,...,0.314123,0.224147,0.295284,0.165086,0.279535,0.184640,0.148243,0.349828,0.120639,0.199725


In [ ]:
#data partitioning

train <- fnor[1:469, ]
test <- fnor[470: 569, ]
train_label <- f[1: 469, 2]
test_label <- f[470: 569, 2]

In [10]:
# Split normalized features
train = dfnor.iloc[:469, :]
test = dfnor.iloc[469:569, :]

# Split labels from original data (column 2, which is index 1 in Python)
train_label = df.iloc[:469, 1]
test_label = df.iloc[469:569, 1]


In [ ]:
# train
p <- knn(train, test, train_label, k=3)

In [11]:
# Initialize the k-NN classifier with k=3
knn = KNeighborsClassifier(n_neighbors=3)

# Fit on training data
knn.fit(train, train_label)

# Predict on test data
p = knn.predict(test)


In [ ]:
# Confusion Matrix
t <- table(actual = test_label,predicted = p)
print(t)

      predicted
actual  B  M
     B 50  1
     M  3 46


In [13]:
# Generate confusion matrix
t = confusion_matrix(test_label, p)

# Optional: Display with row/column labels for clarity
t_df = pd.DataFrame(t, index=knn.classes_, columns=knn.classes_)

print(t_df)


    B   M
B  58   1
M   4  37


In [ ]:
TN <- t[1,1]
TP <- t[2,2]
FN <- t[2,1]
FP <- t[1,2]

In [14]:
# If t is a numpy array from confusion_matrix()
TN = t[0, 0]
TP = t[1, 1]
FN = t[1, 0]
FP = t[0, 1]


In [ ]:
accuracy <- (TN + TP) / (TN +  TP + FN + FP )
cat("Accuracy:", accuracy)

Accuracy: 0.96

In [15]:
accuracy =(TN + TP) / (TN +  TP + FN + FP )
print("Accuracy:", accuracy)

Accuracy: 0.95


In [ ]:
sensitivity <- TP / (TP + FN) 
cat("Sensitivity:", sensitivity)

Sensitivity: 0.9387755

In [16]:
sensitivity = TP / (TP + FN) 
print("Sensitivity:", sensitivity)

Sensitivity: 0.9024390243902439


In [ ]:
specificity <- TN / (TN + FP)
cat("Specificity:", specificity)

Specificity: 0.9803922

In [17]:
specificity = TN / (TN + FP)
print("Specificity:", specificity)

Specificity: 0.9830508474576272


In [ ]:
precision <- TP / (TP + FP)
cat("Precision:", precision)

Precision: 0.9787234

In [18]:
precision = TP / (TP + FP)
print("Precision:", precision)

Precision: 0.9736842105263158
